# Pylops - torch operator

### Author: M.Ravasi

In this notebook I will show how to use the `TorchOperator` to mix and match pylops and pytorch operators into an AD-friendy chain of operations

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import warnings
#warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import torch
import torch.nn as nn

from torch.autograd import gradcheck
from pylops.TorchOperator import TorchOperator
from pylops.basicoperators import *

Single batch

In [2]:
nx, ny = 10, 6
x0 = torch.arange(nx, dtype=torch.double, requires_grad=True)

# Forward
A = np.random.normal(0., 1., (ny, nx))
Aop = TorchOperator(MatrixMult(A))
y = Aop.apply(torch.sin(x0))

# AD
v = torch.ones(ny, dtype=torch.double)
y.backward(v, retain_graph=True)
adgrad = x0.grad

# Analytical
At = torch.from_numpy(A)
#J = (At * torch.cos(x0))
J = (At * torch.cos(x0))
print(J.shape)
anagrad = torch.matmul(J.T, v)

print('Input: ', x0)
print('AD gradient: ', adgrad)
print('Analytical gradient: ', anagrad)

# Grad check
input = (torch.arange(nx, dtype=torch.double, requires_grad=True),
         Aop.matvec, Aop.rmatvec, Aop.device)
test = gradcheck(Aop.Top, input, eps=1e-6, atol=1e-4)
print(test)

torch.Size([6, 10])
Input:  tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=torch.float64,
       requires_grad=True)
AD gradient:  tensor([-0.6358,  0.3454, -0.8364, -0.7692, -3.0928, -0.3300,  1.9894, -0.1716,
        -0.1775, -1.5061], dtype=torch.float64)
Analytical gradient:  tensor([-0.6358,  0.3454, -0.8364, -0.7692, -3.0928, -0.3300,  1.9894, -0.1716,
        -0.1775, -1.5061], dtype=torch.float64, grad_fn=<MvBackward>)
True


Multi batch, we should get here to sum of gradients

In [3]:
nbatch, nx, ny = 5, 3, 6
x0 = torch.arange(nbatch * nx, dtype=torch.float).reshape(nbatch, nx)
x0.requires_grad=True

# Forward
A = np.random.normal(0., 1., (ny, nx)).astype(np.float32)
Aop = TorchOperator(MatrixMult(A), batch=True)
y = Aop.apply(torch.sin(x0))

# AD
v = torch.ones((nbatch, ny), dtype=torch.float32)
y.backward(v, retain_graph=True)
adgrad = x0.grad
print('AD gradient: ', adgrad)

# Analytical
x0.grad.data.zero_()
At = torch.from_numpy(A)
Lin = nn.Linear(nx, ny, bias=False)
Lin.weight.data[:] = At.float()
y1 = Lin(torch.sin(x0))
y1.backward(v, retain_graph=True)
anagrad = x0.grad

print('Analytical gradient: ', anagrad)

AD gradient:  tensor([[-2.4999, -0.2820, -0.6562],
        [ 2.4749,  0.3411,  0.4473],
        [-2.4003, -0.3934, -0.2294],
        [ 2.2777,  0.4379,  0.0070],
        [-2.1096, -0.4736,  0.2156]])
Analytical gradient:  tensor([[-2.4999, -0.2820, -0.6562],
        [ 2.4749,  0.3411,  0.4473],
        [-2.4003, -0.3934, -0.2294],
        [ 2.2777,  0.4379,  0.0070],
        [-2.1096, -0.4736,  0.2156]])


In [4]:
nbatch, nx, ny = 5, 3, 6
x0 = torch.arange(nbatch*nx, dtype=torch.float).reshape(nbatch, nx).requires_grad_()

# Forward
A = np.random.normal(0., 1., (ny, nx)).astype(np.float32)
Aop = TorchOperator(MatrixMult(A), batch=True)
y = Aop.apply(torch.sin(x0))
l = torch.mean(y**2)
l.backward()
adgrad = x0.grad
print('AD gradient: ', adgrad)

# Analytical
x1 = torch.arange(nbatch*nx, dtype=torch.float).reshape(nbatch, nx).requires_grad_()
At = torch.from_numpy(A)
Lin = nn.Linear(nx, ny, bias=False)
Lin.weight.data[:] = At.float()
y1 = Lin(torch.sin(x1))
l1 = torch.mean(y1**2)
l1.backward()
anagrad = x1.grad

print('Analytical gradient: ', anagrad)

AD gradient:  tensor([[-0.0396,  0.0809, -0.0171],
        [-0.0822,  0.0764, -0.0187],
        [-0.1198,  0.0615, -0.0130],
        [-0.1494,  0.0376, -0.0005],
        [-0.1688,  0.0064,  0.0179]])
Analytical gradient:  tensor([[-0.0396,  0.0809, -0.0171],
        [-0.0822,  0.0764, -0.0187],
        [-0.1198,  0.0615, -0.0130],
        [-0.1494,  0.0376, -0.0005],
        [-0.1688,  0.0064,  0.0179]])
